In [3]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [4]:
adeft_grounder = AdeftGrounder()

In [5]:
shortforms = ['PI']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [6]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [7]:
list(zip(longforms, counts))

[('phosphatidylinositol', 1476),
 ('propidium iodide', 1078),
 ('phosphoinositide', 400),
 ('protease inhibitor', 372),
 ('pulsatility index', 120),
 ('post infection', 84),
 ('proteasome inhibitor', 64),
 ('proliferation index', 61),
 ('pistillata', 52),
 ('phosphatidyl inositol', 52),
 ('plaque index', 44),
 ('pyrrole imidazole', 43),
 ('pneumatosis intestinalis', 34),
 ('pelvic incidence', 33),
 ('phagocytic index', 33),
 ('pars intercerebralis', 32),
 ('polyimide', 32),
 ('post inoculation', 31),
 ('postinfection', 27),
 ('proactive interference', 26),
 ('pulse interval', 26),
 ('persistently infection', 26),
 ('phosphoinositol', 26),
 ('povidone iodine', 23),
 ('proteinase inhibitor', 23),
 ('perfusion index', 23),
 ('prognostic index', 20),
 ('proliferative index', 20),
 ('parallel imaging', 17),
 ('prescriptive information', 15),
 ('phosphatidylinositide', 15),
 ('principal investigated', 14),
 ('pulmonary insufficiency', 13),
 ('prostate index', 13),
 ('pathogen inactivation', 

In [8]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [9]:
names

{'CHEBI:CHEBI:28874': 'phosphatidylinositol',
 'CHEBI:CHEBI:51240': 'propidium iodide',
 'CHEBI:CHEBI:18179': 'phosphoinositide',
 'CHEBI:CHEBI:35222': 'inhibitor',
 'MESH:D020481': 'Index',
 'MESH:D007239': 'Infections',
 'CHEBI:CHEBI:52726': 'proteasome inhibitor',
 'CHEBI:CHEBI:16749': '1-phosphatidyl-1D-myo-inositol',
 'MESH:D015994': 'Incidence',
 'CHEBI:CHEBI:53232': 'polyimide macromolecule',
 'CHEBI:CHEBI:8347': 'Povidone-iodine',
 'MESH:D000081282': 'Perfusion Index',
 'MESH:D014947': 'Wounds and Injuries',
 'CHEBI:CHEBI:17606': 'diiodine',
 'MESH:D010188': 'Exocrine Pancreatic Insufficiency',
 'MESH:D011384': 'Proinsulin',
 'MESH:D010743': 'Phospholipids',
 'MESH:D007267': 'Injections',
 'CHEBI:CHEBI:17191': 'L-isoleucine',
 'CHEBI:CHEBI:50847': 'immunological adjuvant',
 'MESH:C559491': '2-(4-isobutylphenyl)propionic acid 4-(diethoxyphosphoryloxy)butyl ester',
 'MESH:D007422': 'Intestines',
 'MESH:D007319': 'Sleep Initiation and Maintenance Disorders',
 'CHEBI:CHEBI:53405': 

In [10]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-11-11 19:43:02] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [11]:
result = [grounding_map, names, pos_labels]

In [12]:
result

[{'pain intensity': 'ungrounded',
  'pain interference': 'ungrounded',
  'pallidal index': 'ungrounded',
  'pam interaction': 'ungrounded',
  'pancreatic insufficiency': 'MESH:D010188',
  'parainflammation': 'ungrounded',
  'parallel imaging': 'ungrounded',
  'parametrial involved': 'ungrounded',
  'parent incarceration': 'ungrounded',
  'pars intercerebralis': 'ungrounded',
  'pars intermedia': 'ungrounded',
  'paternity index': 'ungrounded',
  'path integrity': 'ungrounded',
  'pathogen inactivation': 'ungrounded',
  'pattern identification': 'ungrounded',
  'pausing index': 'ungrounded',
  'peak intensity': 'ungrounded',
  'peak interval': 'ungrounded',
  'pelvic incidence': 'ungrounded',
  'pentamidine isethionate': 'CHEBI:CHEBI:7977',
  'performed improvement': 'ungrounded',
  'performed index': 'ungrounded',
  'performed indicated': 'ungrounded',
  'perfusion index': 'MESH:D000081282',
  'period': 'ungrounded',
  'peripheral iridectomy': 'MESH:D032801',
  'peritoneal implant': 'u

In [46]:
grounding_map, names, pos_labels = [{'pain intensity': 'ungrounded',
  'pain interference': 'ungrounded',
  'pallidal index': 'ungrounded',
  'pam interaction': 'ungrounded',
  'pancreatic insufficiency': 'MESH:D010188',
  'parainflammation': 'ungrounded',
  'parallel imaging': 'ungrounded',
  'parametrial involved': 'ungrounded',
  'parent incarceration': 'ungrounded',
  'pars intercerebralis': 'ungrounded',
  'pars intermedia': 'ungrounded',
  'paternity index': 'ungrounded',
  'path integrity': 'ungrounded',
  'pathogen inactivation': 'ungrounded',
  'pattern identification': 'ungrounded',
  'pausing index': 'ungrounded',
  'peak intensity': 'ungrounded',
  'peak interval': 'ungrounded',
  'pelvic incidence': 'ungrounded',
  'pentamidine isethionate': 'CHEBI:CHEBI:7977',
  'performed improvement': 'ungrounded',
  'performed index': 'ungrounded',
  'performed indicated': 'ungrounded',
  'perfusion index': 'MESH:D000081282',
  'period': 'ungrounded',
  'peripheral iridectomy': 'MESH:D032801',
  'peritoneal implant': 'ungrounded',
  'permanent implant': 'ungrounded',
  'permeability index': 'ungrounded',
  'peroxide intermediate': 'ungrounded',
  'peroxidizability index': 'ungrounded',
  'persistently index': 'ungrounded',
  'persistently infection': 'ungrounded',
  'phagocytic index': 'ungrounded',
  'phatidylinositol': 'CHEBI:CHEBI:28874',
  'phospha tidylinositol': 'CHEBI:CHEBI:28874',
  'phosphatidyl inositol': 'CHEBI:CHEBI:28874',
  'phosphatidyl myo inositol': 'CHEBI:CHEBI:28874',
  'phosphatidylinositide': 'CHEBI:CHEBI:28874',
  'phosphatidylinositol': 'CHEBI:CHEBI:28874',
  'phospho ibuprofen': 'MESH:C559491',
  'phospho inositide': 'CHEBI:CHEBI:28874',
  'phosphoinositide': 'CHEBI:CHEBI:28874',
  'phosphoinositide hydrolysis': 'CHEBI:CHEBI:28874',
  'phosphoinositol': 'CHEBI:CHEBI:28874',
  'phospholipids': 'MESH:D010743',
  'phosphotidylinositol': 'CHEBI:CHEBI:28874',
  'phosphotyrosine interaction': 'ungrounded',
  'photoinactivation': 'ungrounded',
  'photoinitiator': 'ungrounded',
  'physical inactivity': 'ungrounded',
  'phytochemical index': 'ungrounded',
  'phytosanitary irradiation': 'ungrounded',
  'pi for 4 h': 'ungrounded',
  'pistillata': 'ungrounded',
  'pkr inhibitor': 'ungrounded',
  'place invariants': 'ungrounded',
  'placental insufficiency': 'MESH:D010927',
  'plant introduction': 'ungrounded',
  'plantar incision': 'ungrounded',
  'plaque index': 'ungrounded',
  'plasma insulin': 'HGNC:6081',
  'plasmin inhibitor': 'ungrounded',
  'pma and calcium ionophore': 'ungrounded',
  'pma and ionomycin': 'ungrounded',
  'pma ionomycin': 'ungrounded',
  'pneumatosis intestinalis': 'ungrounded',
  'point': 'ungrounded',
  'polar index': 'ungrounded',
  'polydispersity index': 'ungrounded',
  'polyfunctionality index': 'ungrounded',
  'polyimide': 'ungrounded',
  'polyinosinic acid': 'CHEBI:CHEBI:76777',
  'polyisoprene': 'CHEBI:CHEBI:53405',
  'polyprenyl immunostimulant': 'ungrounded',
  'ponderal index': 'ungrounded',
  'positive index': 'ungrounded',
  'positive intervention': 'ungrounded',
  'post immune': 'ungrounded',
  'post infection': 'ungrounded',
  'post injection': 'ungrounded',
  'post injury': 'ungrounded',
  'post inoculation': 'ungrounded',
  'post inspiratory': 'ungrounded',
  'post institutionalized': 'ungrounded',
  'post irradiation': 'ungrounded',
  'posterior insula': 'ungrounded',
  'postinfection': 'ungrounded',
  'postinoculation': 'ungrounded',
  'povidone iodine': 'ungrounded',
  'pr inhibitor': 'ungrounded',
  'preference index': 'ungrounded',
  'prescriptive information': 'ungrounded',
  'pressure injury': 'ungrounded',
  'primary index': 'ungrounded',
  'primary insomnia': 'ungrounded',
  'principal investigated': 'ungrounded',
  'pristinamycin i': 'ungrounded',
  'proactive interference': 'ungrounded',
  'probability interval': 'ungrounded',
  'progestin independent': 'ungrounded',
  'prognostic index': 'ungrounded',
  'proinflammatory': 'ungrounded',
  'proinsulin': 'MESH:D011384',
  'proliferation index': 'ungrounded',
  'proliferative index': 'ungrounded',
  'prolonged infusion': 'ungrounded',
  'propidium iodide': 'CHEBI:CHEBI:51240',
  'propidium iodine': 'CHEBI:CHEBI:51240',
  'propionate acid': 'ungrounded',
  'proportional integrity': 'ungrounded',
  'prostate index': 'ungrounded',
  'prostate inflammation': 'ungrounded',
  'prostate interstitial brachytherapy': 'ungrounded',
  'protease inhibitor': 'MESH:D011480',
  'protease inhibitor cocktail': 'MESH:D011480',
  'proteasome inhibition': 'CHEBI:CHEBI:52726',
  'proteasome inhibitor': 'CHEBI:CHEBI:52726',
  'protective index': 'ungrounded',
  'protein inclusions': 'ungrounded',
  'proteinase inhibitor': 'MESH:D011480',
  'proximal intestine': 'ungrounded',
  'psychological intervention': 'ungrounded',
  'psychophysiological insomnia': 'ungrounded',
  'pulmonary insufficiency': 'ungrounded',
  'pulsatility index': 'ungrounded',
  'pulsatility indicated': 'ungrounded',
  'pulse interval': 'ungrounded',
  'pulse inversion': 'ungrounded',
  'punishment insensitivity': 'ungrounded',
  'pyrrole imidazole': 'ungrounded'},
 {'MESH:D010188': 'Exocrine Pancreatic Insufficiency',
  'CHEBI:CHEBI:7977': 'pentamidine isethionate',
  'MESH:D000081282': 'Perfusion Index',
  'MESH:D032801': 'Iridectomy',
  'CHEBI:CHEBI:28874': 'phosphatidylinositol',
  'MESH:C559491': '2-(4-isobutylphenyl)propionic acid 4-(diethoxyphosphoryloxy)butyl ester',
  'MESH:D010743': 'Phospholipids',
  'MESH:D010927': 'Placental Insufficiency',
  'HGNC:6081': 'INS',
  'CHEBI:CHEBI:76777': 'poly(inosinic acid)',
  'CHEBI:CHEBI:53405': 'poly(isoprene) macromolecule',
  'MESH:D011384': 'Proinsulin',
  'CHEBI:CHEBI:51240': 'propidium iodide',
  'MESH:D011480': 'Protease Inhibitors',
  'CHEBI:CHEBI:52726': 'proteasome inhibitor'},
 ['CHEBI:CHEBI:28874', 'CHEBI:CHEBI:51240', 'MESH:D011480']]

In [47]:
excluded_longforms = ['pi for 4h', 'pkr inhibitor', 'pr inhibitor']

In [48]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [49]:
additional_entities = {}

In [50]:
unambiguous_agent_texts = {}

In [51]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [52]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [53]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [54]:
intersection1

[]

In [55]:
intersection2

[]

In [56]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [57]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [58]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-11 20:16:01] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-11 20:17:41] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9430661655441552 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [59]:
classifier.stats

{'label_distribution': {'MESH:D011480': 316,
  'ungrounded': 821,
  'CHEBI:CHEBI:28874': 1714,
  'MESH:D010743': 11,
  'MESH:D011384': 11,
  'CHEBI:CHEBI:51240': 777,
  'CHEBI:CHEBI:7977': 4,
  'HGNC:6081': 3,
  'MESH:D010188': 9,
  'CHEBI:CHEBI:76777': 3,
  'CHEBI:CHEBI:53405': 4,
  'CHEBI:CHEBI:52726': 57,
  'MESH:D000081282': 16,
  'MESH:C559491': 2,
  'MESH:D010927': 3,
  'MESH:D032801': 2},
 'f1': {'mean': 0.943066, 'std': 0.006887},
 'precision': {'mean': 0.938874, 'std': 0.008594},
 'recall': {'mean': 0.948345, 'std': 0.008929},
 'MESH:D010188': {'f1': {'mean': 0.366667, 'std': 0.305505},
  'pr': {'mean': 0.4, 'std': 0.374166},
  'rc': {'mean': 0.466667, 'std': 0.452155}},
 'MESH:C559491': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'HGNC:6081': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'MESH:D032801': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.

In [60]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [61]:
disamb.dump(model_name, results_path)

In [62]:
print(disamb.info())

Disambiguation model for PI

Produces the disambiguations:
	2-(4-isobutylphenyl)propionic acid 4-(diethoxyphosphoryloxy)butyl ester	MESH:C559491
	Exocrine Pancreatic Insufficiency	MESH:D010188
	INS	HGNC:6081
	Iridectomy	MESH:D032801
	Perfusion Index	MESH:D000081282
	Phospholipids	MESH:D010743
	Placental Insufficiency	MESH:D010927
	Proinsulin	MESH:D011384
	Protease Inhibitors*	MESH:D011480
	pentamidine isethionate	CHEBI:CHEBI:7977
	phosphatidylinositol*	CHEBI:CHEBI:28874
	poly(inosinic acid)	CHEBI:CHEBI:76777
	poly(isoprene) macromolecule	CHEBI:CHEBI:53405
	propidium iodide*	CHEBI:CHEBI:51240
	proteasome inhibitor	CHEBI:CHEBI:52726

Class level metrics:
--------------------
Grounding                                                              	Count	F1     
                                                   phosphatidylinositol*	1714	0.96107
                                                             Ungrounded	 821	0.90215
                                                       propid

In [63]:
model_to_s3(disamb)

In [30]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [31]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:10967']

In [36]:
texts[3]

'The non-neuronal monoamine transporters (OCT1, OCT2, EMT, and PMAT) play a key role in the clearance of monoamines from extracellular compartments. In a previous report we described endometrial distribution and cyclic variation of the vesicular monoamine transporter (VMAT2) mRNA and the neuronal norepinephrine transporter (NET) mRNA. In the present study we used in situ hybridization, real-time PCR and immunohistochemistry to reveal tissue distribution and cyclic variation of mRNA for the non-neuronal monoamine transporters in the human endometrium and early pregnancy decidua. We found that non-neuronal monoamine transporters are predominantly expressed in the stroma. The plasma membrane monoamine transporter (PMAT) mRNA expression peaked in the proliferative phase, whereas the extra-neuronal monoamine transporter (EMT) mRNA expression peaked in the secretory phase. The organic cation transporter 2 (OCT2) mRNA expression was exclusively detected in few scattered stromal cells and OCT1